In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
lr = 3e-4
eval_iters = 250

In [ ]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100]) 

tensor([28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36, 11,
         1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32, 39,
        42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,
         1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1, 39,
        50,  9,  1, 39, 50, 37, 25,  1, 39, 30])


In [5]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    xi = torch.randint(len(data) - block_size, (batch_size,)) # random starting indices
    # print(f"Training Integers: {xi}")
    # input batch
    x = torch.stack([data[i:i+block_size] for i in xi])
    # target batch    
    y = torch.stack([data[i+1:i+1+block_size] for i in xi])
    x, y = x.to(device), y.to(device) # send to cuda -> parallelize
    return x, y

x, y = get_batch('train')
print("Input batch:")
print(x)
print("Target batch:")
print(y)    

Input batch:
tensor([[58, 57,  0, 54, 60, 54, 62, 67],
        [ 1, 76, 54, 78,  1, 34, 62, 66],
        [58, 66,  1, 54, 67, 57,  1, 34],
        [62, 56, 64,  1, 68, 74, 73,  1]], device='cuda:0')
Target batch:
tensor([[57,  0, 54, 60, 54, 62, 67,  1],
        [76, 54, 78,  1, 34, 62, 66,  1],
        [66,  1, 54, 67, 57,  1, 34, 62],
        [56, 64,  1, 68, 74, 73,  1, 76]], device='cuda:0')


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # type: ignore
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # type: ignore
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # type: ignore
    return out

In [7]:
# basic bigram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B=batch size, T=time, C=channels (vocab size)
            
            # reshape logits and targets for cross-entropy loss
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)  # get predictions
            logits = logits[:, -1, :] # (B, C) last time step
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size).to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)        


GDg'ZW5h
)JY;sD0LGZ3
_Oe*3!k]5y5?-wFLs?RO1."0TL2h
HXvX:t7Z&kD?Kmq,3iBWyCO6faKJjMC .SNN!HFF8U7baT9r!dqh_;VQ?)zgMpfHxOIm)&EFoL:NqJM:tk?*IL)SIm-Dq]cq:'CKBDgR7(MNT]5F(OB.R!'MmiO;3 PN54OUL;rFl35GS68M[qu(8P,F&xT o
p&EK?s?Fnb.9"9Lq
IT)54]5mLhD:Z.]2CWY[3YNZ&KE2x;&-1O410.KQ?EayxJP]A'_Z,"evSJ]Gv!?yCjh
Kqe*8b.
nW6V_Sssr )Ps0-sOp,"w'Zqd"ey1pQr_b.o1D&l-Aug0PZgG&T)J:)F7ZvupcHY1W'E 3P,9-1H1Dt:*6i?)J3z0SE&*9DSadl6JoJIo1Wt7JH."ac6V6_8n3c'm4qJ&zfj*T_b(cx:ZWCk8d"z!37Dg]_PSNaR7mBY1pP.uVD4ty9Ve1YTXako;0mQYPej*3AG?uJ


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # type: ignore
    optimizer.step()
print(loss.item()) # type: ignore

step: 0, train loss: 4.4338, val loss: 4.4512
step: 250, train loss: 4.3809, val loss: 4.3759
step: 500, train loss: 4.3483, val loss: 4.3138
step: 750, train loss: 4.2825, val loss: 4.2885
4.475279331207275


In [12]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars) 


im4d6MUM15(F(6l-DEs6*(p8E_87_O."vYfSEf?*7*gK
w,7x:f&yNm'RK9yGO-1g,FUAJK]2NWnHxVr1n('Tr-GnEq*&(uvqe8Pe'qJx(powVh
B PBC;B6&?[ei[&_b.5C90qVfouRToTadMdl[E-wPp9-s7yh?rg
)&kH_LG b;Z5F?"E
we b?hexxUZ8SQL*i9L*76RhJ1qWq-T-Mtrb.gFXw_KF('Tgn,"M5mZ&R_;Yn."3
;Y1lJ)6bTH;(BE!av"3zb;)62e1M9gvk;Q 98 nB2xKjIx[E_7xKhen*OftxMzZ9e-1qJUk8Mzgl-"5!k8j!p2'EfWP.PQ841q]NhFnGDnBZ90H"9Lzh
rjh
5cgJU0brivswe0L;dtujD0
;Z5?q**ctHk"5zvu-1zWg0-"g?iWNE-Sgy5ef1ee,&tmLoTM8?J1WJ]5j2ALU((c?AC)Y(?nS3 v?tye*Nxt
oANc].&
iu:MH;TG2q1yZ((mk
